In [8]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.10.0'

In [15]:
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding, CuDNNGRU
from tensorflow.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
mark_start = 'ssss ' #decoderın çıktı üretilmesi için start token gerekiyor 
mark_end = ' eeee'

In [17]:
data_src = []
data_dest = []

In [18]:
for line in open('tur.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')
    
    tr_text = mark_start + tr_text + mark_end           #tur.text veri setindeki ingilizce ve türkçe veriler ayrıştırılır ve turkçe verilerin başına ve sonuna end start markları eklenir. 
    
    data_src.append(en_text)
    data_dest.append(tr_text)

In [19]:
data_src[100]  #ayrıştırma işleminde sorun olup olmadığı kontrol edilir.

'I drove.'

In [ ]:
data_dest[100]

'ssss Araba sürdüm. eeee'

In [ ]:
data_src[200000]

'Can you see anything missing?'

In [ ]:
data_dest[200000]

'ssss Eksik bir şey görebiliyor musun? eeee'

In [ ]:
len(data_src) #ingilizce verilerin uzunluğu kontrol edilir.

473035

In [ ]:
class TokenizerWrap(Tokenizer):  #Daha sonra yapılacak olan kullanımlarada kolaylık sağlanması amacıyla sınıflandırma kullanılmıştır.
    def __init__(self, texts, padding, reverse=False, num_words=None):  #constructer kullanılarak tanımlanan her sınıf değeri için bir alt fonksiyon ile belirlenen parametrelerin uygun olması koşuluyla tokenizer işlemi gerçekleşir.
        Tokenizer.__init__(self, num_words=num_words)
        
        self.fit_on_texts(texts)
        
        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys()))
        
        self.tokens = self.texts_to_sequences(texts)
        
        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens] #padding işlemi uygulanır. Padding işlemi uygulanırken vectör oluşturulma durumunda cümlelerin ters dönme durumları göz önüne alınarak post ve pre olarak uygulanır.
            truncating = 'pre'
        else:
            truncating = 'post'
            
        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens) #İşlenicek cümleler uzunluklarının aynı olması gerektiği için tüm cümleleerin ortalama uzunlarına göre ideal uzunluk değeri hesaplanarak tüm cümleler o uzunluğa uygun olarak kesilir yada boş dolarak doldurulur.
        self.max_tokens = int(self.max_tokens)
        
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)
        
    def token_to_word(self, token):
        word = ' ' if token == 0 else self.index_to_word[token]  #Decoderdan alınan tokenlerin kelimelere dönüştürme işleminin yapılması için uygun fonksiyon hazırlanır  
        return word
    
    def tokens_to_string(self, tokens):
        words = [self.index_to_word[token] for token in tokens if token != 0]   
        text = ' '.join(words)
        return text
    
    def text_to_tokens(self, text, padding, reverse=False):
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)
        
        if reverse:                                             #Üst tarafta hazırlanan padding parametrelerinde olduğu gibi uygun post veya pre işlemine ve uzunluk belirleme işlemine göre uygun tokenizer işlemi yapılır. 
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'
            
        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)
        
        return tokens

In [ ]:
tokenizer_src = TokenizerWrap(texts=data_src,  #Tokenleştirme işlemi yapılır.
                              padding='pre',
                              reverse=True,
                              num_words=None)

In [ ]:
tokenizer_dest = TokenizerWrap(texts=data_dest,
                              padding='post',
                              reverse=False,
                              num_words=None)

In [ ]:
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape)
print(tokens_dest.shape)

(473035, 11)
(473035, 10)


In [ ]:
tokens_dest[200000] #Tokenleştirilme işlemi kontrol edilir.

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0,    0])

In [ ]:
tokenizer_dest.tokens_to_string(tokens_dest[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [ ]:
tokens_src[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39])

In [ ]:
tokenizer_src.tokens_to_string(tokens_src[200000])

'missing anything see you can'

In [ ]:
data_src[200000]

'Can you see anything missing?'

In [ ]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start   #Start ve end tokenleri kullanım sıkılığından dolayı 1 ve 2 değerlerini alır.

1

In [ ]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end

2

In [ ]:
encoder_input_data = tokens_src #encoder input data tanımlanır.

In [ ]:
decoder_input_data = tokens_dest[:, :-1]
decoder_output_data = tokens_dest[:, 1:]  

In [ ]:
encoder_input_data[200000]

array([   0,    0,    0,    0,    0,    0, 1028,  113,   95,    5,   39])

In [ ]:
decoder_input_data[200000]

array([   1, 2391,    4,   18, 4127,   48,    2,    0,    0])

In [ ]:
decoder_output_data[200000]

array([2391,    4,   18, 4127,   48,    2,    0,    0,    0])

In [ ]:
tokenizer_dest.tokens_to_string(decoder_input_data[200000])

'ssss eksik bir şey görebiliyor musun eeee'

In [ ]:
tokenizer_dest.tokens_to_string(decoder_output_data[200000])

'eksik bir şey görebiliyor musun eeee'

In [ ]:
num_encoder_words = len(tokenizer_src.word_index)
num_decoder_words = len(tokenizer_dest.word_index)

In [ ]:
num_encoder_words

21315

In [ ]:
num_decoder_words

94058

In [ ]:
embedding_size = 100

In [ ]:
word2vec = {}
with open('glove.6B.100d.txt', encoding='UTF-8') as f: #Daha önceden hazırlanmış glove vectörüden alınan vectörler ve kelimeler  ayrılarak bir sözlük içinde saklanır.
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

In [ ]:
embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size)) #Alınan token değerlerini vectöre dönüştürmek ve bu vectörlere göre layerları eğitmek için araya bir embedding  layer eklememiz gerekiyor.
for word, i in tokenizer_src.word_index.items():
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(21315, 100)

In [ ]:
encoder_input = Input(shape=(None,), name='encoder_input')

In [ ]:
encoder_embedding = Embedding(input_dim=num_encoder_words, #Oluşturulan bu layerın encodera gelen kelimeleri alması ve 100 vectör uzunluğuna göre çevirmesi gerekiyor.
                              output_dim=embedding_size,
                              weights=[embedding_matrix],
                              trainable=True,
                              name='encoder_embedding')

In [ ]:
state_size = 256

In [ ]:
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)  #Son layer  çıktı verdiği için son layer hariç diğer layerların eğitilebilr olması gerekiyor 
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False)

In [ ]:
def connect_encoder():
    net = encoder_input
    
    net = encoder_embedding(net)  #Oluşturulan embedding ve encoder layerlarını birbirine bağlalıyoruz.
    
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    
    encoder_output = net
    
    return encoder_output

In [ ]:
encoder_output = connect_encoder()

In [ ]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')

In [ ]:
decoder_input = Input(shape=(None,), name='decoder_input')

In [ ]:
decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [ ]:
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)  #Son layerdan sonra bir dens edaha ekleyeceğimiz için hepsi eğitilebilir olabilir.
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True)

In [ ]:
decoder_dense = Dense(num_decoder_words,
                      activation='linear', #En son katmana bir lineer layer ekleyerek çıktı alıyoruz.
                      name='decoder_output')

In [ ]:
def connect_decoder(initial_state):
    net = decoder_input
    
    net = decoder_embedding(net)
    
    net = decoder_gru1(net, initial_state=initial_state)
    net = decoder_gru2(net, initial_state=initial_state)
    net = decoder_gru3(net, initial_state=initial_state)
    
    decoder_output = decoder_dense(net)
    
    return decoder_output

In [ ]:
decoder_output = connect_decoder(initial_state=encoder_output)

model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output]) #Modellerin eğitim sırasında uygun sırada verileri alıp eğitilmesi için modellerin çıktılarının ve girdilerinin birbirine bağlı olması gerekiyor.

In [ ]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output])

In [ ]:
decoder_output = connect_decoder(initial_state=decoder_initial_state)

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output])

In [ ]:
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred) #Tensorflow dan kayn aklı aldığın bir hatadan dolayı loss fonksiyonun kendim oluşturdum.
    loss_mean = tf.reduce_mean(loss)
    return loss_mean

In [ ]:
optimizer = RMSprop(lr=1e-3) #Uygun optimizer değeri verildi.

In [ ]:
decoder_target = tf.placeholder(dtype='int32', shape=(None,None))

model_train.compile(optimizer=optimizer,
                    loss=sparse_cross_entropy,
                    target_tensors=[decoder_target])

In [55]:
path_checkpoint = 'checkpoint.keras'   #Daha sonradan kullanım kolaylığı olması açısından kayıt işlemi eklendi.
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True)

In [56]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)

In [57]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data} #İnputlar ve outputlar belirlendi.

In [58]:
y_data = {'decoder_output': decoder_output_data}

In [59]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256,  #Kullanılan sisteme göre batch_size ve epochs değerleri değiştirilebilir.
                epochs=2,
                callbacks=[checkpoint])

C:\Users\Maynard\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [60]:
def translate(input_text, true_output_text=None):   #Bu fonksiyonda deceodera verilen tokenlere göre alınan initial_state değeri daha önceden hazırlanmış tokens_to_text fonksiyonu ile çevrilme işlemi yapılır.  
    input_tokens = tokenizer_src.text_to_tokens(text=input_text,
                                                reverse=True,
                                                padding='pre')
    
    initial_state = model_encoder.predict(input_tokens)     
    
    max_tokens = tokenizer_dest.max_tokens
    
    decoder_input_data = np.zeros(shape=(1, max_tokens), dtype=np.int)
    
    token_int = token_start
    output_text = ''
    count_tokens = 0
    
    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int
        x_data = {'decoder_initial_state': initial_state, 'decoder_input': decoder_input_data}
        
        decoder_output = model_decoder.predict(x_data)
        
        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)
        
        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += ' ' + sampled_word
        count_tokens += 1
        
    print('Input text:')
    print(input_text)
    print()
    
    print('Translated text:')
    print(output_text)
    print()
    
    if true_output_text is not None:
        print('True output text:')
        print(true_output_text)
        print()

In [65]:
translate(input_text=data_src[400000], true_output_text=data_dest[400000]) #Çıktılar orjinal değerlere göre kontrol edilir.

Input text:
You are telling it second hand, aren't you?

Translated text:
 onu ikinci balık söylüyorsun değil mi eeee

True output text:
ssss Onu dolaylı olarak anlatıyorsun, değil mi? eeee



In [68]:
translate(input_text='Which road leads to the airport?')

Input text:
Which road leads to the airport?

Translated text:
 hangi yol havaalanına gider eeee

